In [128]:
import numpy as np

In [129]:
board = np.asarray([["-"]*8]*8)
print(board.shape)

(8, 8)


In [130]:
state_tree = []
state_tree_dict = []

In [131]:
def initialize_board():
    for i in range(len(board)):
        for j in range(len(board[0])):
            if((i+j)%2 == 0):
                board[i][j] = "X"
            else:
                board[i][j] = "O"

In [132]:
initialize_board()

In [133]:
def remove_piece(board_state, row,col):
    if(0 <= row and row < len(board_state) and 0 <= col and col < len(board_state[0])):
        board_state[row][col] = '.'
        return board_state
    else:
        print("invalid remove\n")
        return None

In [134]:
def move_piece(board_state, team, srow, scol, erow, ecol):
    if(0 <= srow and srow < len(board_state) and 0 <= scol and scol < len(board_state[0]) and 0 <= erow and erow < len(board_state) and 0 <= ecol and ecol < len(board_state[0])):
        board_state[srow][scol] = '.'
        # vertical move
        if(scol == ecol):
            if(srow < erow):
                for i in range(srow, erow+1, 2):
                    board_state[i+1][scol] = '.'
            else:
                for i in range(srow, erow-1, -2):
                    board_state[i-1][scol] = '.'
        # horizontal move
        elif(srow == erow):
            if(scol < ecol):
                for j in range(scol, ecol+1, 2):
                    board_state[srow][j+1] = '.'
            else:
                for j in range(scol, ecol-1, -2):
                    board_state[srow][j-1] = '.'
        else:
            print("non orthogonal move")
            return None
    else:
        print("move out of range")
        return None
    
    board_state[erow][ecol] = team
    return(board_state)

In [135]:
# heuristic moves_available to choose best move, returns a list of possible board states
def moves_available(team, board_state):
   num_moves = 0
   board_states = []
   a = np.where(board_state == team)
   for i,j in zip(a[0],a[1]):
      # up
      k = i
      while k-2 >= 0:
         bs_cpy = board_state.copy()
         if(board_state[k-2][j] == '.' and board_state[k-1][j] != team and board_state[k-1][j] != '.'):
            num_moves+=1
            board_states.append(move_piece(bs_cpy, team, i,j,k-2,j))
            k-=2
         else:
            break
      # down
      k = i
      while k+2 < len(board_state):
         bs_cpy = board_state.copy()
         if(board_state[k+2][j] == '.' and board_state[k+1][j] != team and board_state[k+1][j] != '.'):
            num_moves+=1
            board_states.append(move_piece(bs_cpy, team, i,j,k+2,j))
            k+=2
         else:
            break
      # left
      k = j
      while k-2 >= 0:
         bs_cpy = board_state.copy()
         if(board_state[i][k-2] == '.' and board_state[i][k-1] != team and board_state[i][k-1] != '.'):
            num_moves+=1
            board_states.append(move_piece(bs_cpy, team, i,j,i,k-2))
            k-=2
         else:
            break
      # right
      k = j
      while k+2 < len(board_state[0]):
         bs_cpy = board_state.copy()
         if(board_state[i][k+2] == '.' and board_state[i][k+1] != team and board_state[i][k+1] != '.'):
            num_moves+=1
            board_states.append(move_piece(bs_cpy, team, i,j,i,k+2))
            k+=2
         else:
            break
   return board_states
      

In [136]:
def print_board(board_state):
    for i in range(len(board_state)):
        str = ""
        for j in range(len(board_state[0])):
            str = str + board_state[i][j] + " "
        print(str)

In [137]:
print_board(board)

X O X O X O X O 
O X O X O X O X 
X O X O X O X O 
O X O X O X O X 
X O X O X O X O 
O X O X O X O X 
X O X O X O X O 
O X O X O X O X 


In [138]:
board = remove_piece(board,4,4)

In [139]:
print_board(board)
print("")
print(len(moves_available('X', board)))

for brd in moves_available('X', board):
    print(len(moves_available('X', brd)))


X O X O X O X O 
O X O X O X O X 
X O X O X O X O 
O X O X O X O X 
X O X O . O X O 
O X O X O X O X 
X O X O X O X O 
O X O X O X O X 

4
3
3
2
2
